# Final Neural Network Model Selection Process

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, r2_score
from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import keras_tuner as kt

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
from  IPython import display

import pathlib
import shutil
import tempfile

print(tf.__version__)

2.7.0


Ensure the random state is consistent:

In [2]:
from numpy.random import seed
from numpy.random import RandomState

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

### Data loading, transformation, and normalization

In [4]:
data = pd.read_csv("./data/nba_final_data.csv")
data = data.sample(frac=1, random_state = 42) # Shuffle data
y = data['PLUS_MINUS_HOME']
X = data.drop(['GAME_ID', "TEAM_ID_HOME", "TEAM_ID_AWAY", "GAME_DATE", "SEASON",
               "PLUS_MINUS_HOME", "MIN_HOME", 'WL_Home_modified'], axis=1)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

### HyperParameter Tuning With Bayesian Optimization: Setup

In [5]:
# Define log dir 
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [6]:
def model_builder(hp):
    model = keras.Sequential()
    
    # hyperparameter boolean for performing dropout 
    dropout = hp.Boolean("dropout") 
    # hyperparameter for percent of units to dropout 
    if dropout:
        drop_percent = hp.Choice("drop_percent", [0.1, 0.25, 0.5])
        
    kernel_regularizer= regularizers.l2(0.001)
    
    # hyperparameter for choice of regularization strength
    regularization = hp.Choice("regularization_strength", [0.0001, 0.001, 0.01])
    
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))
    for i in range(hp.Int("num_layers", 1, 3)):
        hp_units = hp.Int('units', min_value=8, max_value=64, step=4)
        model.add(
            keras.layers.Dense(units=hp_units,
                               activation='elu',
                               kernel_regularizer = regularizers.l2(regularization))
        )
        # Add dropout layer if dropout hyperparameter is True
        if dropout:
            keras.layers.Dropout(drop_percent)
            
    model.add(keras.layers.Dense(1)) # output layer
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss='mse',
            metrics=['mse'])
    return model

In [7]:
def get_callbacks(name):
    return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50),
    tf.keras.callbacks.TensorBoard(logdir/name)
    ]

In [8]:
tuner = kt.BayesianOptimization(model_builder,
                                objective='val_mse',
                                max_trials = 500,
                                seed=42,
                                overwrite=True
                                )

### HyperParameter Tuning With Bayesian Optimization: Search for Parameters
Here, we search for the best set of hyperparameters with the tuner, extract the hyperparemeters, and build a model with the results 

In [9]:
tuner.search(X_train, y_train, epochs=25, validation_split=0.2, callbacks=get_callbacks('tuner'))

Trial 500 Complete [00h 00m 14s]
val_mse: 149.31893920898438

Best val_mse So Far: 147.8925323486328
Total elapsed time: 05h 52m 50s
INFO:tensorflow:Oracle triggered exit


In [10]:
# Get the optimal hyperparameter model
best_hps = tuner.get_best_hyperparameters()[0]

# Build and save it for future use 
best_model = tuner.hypermodel.build(best_hps)

In [11]:
print(f"""
The hyperparameter search is complete.
Num_layers: {best_hps.get('num_layers')}
Num_units: {best_hps.get('units')}
Dropout: {best_hps.get('dropout')}
Dropout rate: {best_hps.get('drop_percent')}
Regularization strength: {best_hps.get("regularization_strength")}
Learning Rate: {best_hps.get('learning_rate')}
""")


The hyperparameter search is complete.
Num_layers: 3
Num_units: 64
Dropout: False
Dropout rate: 0.5
Regularization strength: 0.01
Learning Rate: 0.001



### HyperParameter Tuning With Bayesian Optimization: Refit and Epoch Selection
Now that we have the best hyperparameters, we refit the model. To determine the optimal number of training epochs, we find the epoch number of the best results as determined by mean square error on the validation set. Finally, we rebuild the model from the hyperparameters, specify the number of epochs as determined by the validation mean square error, and refit the model with the full training data set (train + validation). Then, test results are reported in the final cell. 

In [12]:
history = best_model.fit(X_train, y_train,
                            epochs=50,
                            validation_split=0.2,
                            verbose=0,
                            callbacks=get_callbacks('best_model_epoch_selection'))


Epoch: 0, loss:163.5843,  mse:161.7146,  val_loss:155.5696,  val_mse:153.7273,  
..................................................

In [13]:
validation_mse = history.history['val_mse']
min_mse = min(validation_mse)
min_idx = validation_mse.index(min_mse)
num_epochs = min_idx
print(f"Minimum MSE: {min_mse}")

Minimum MSE: 149.58773803710938


In [15]:
best_model = tuner.hypermodel.build(best_hps)
history=best_model.fit(X_train, y_train, epochs=num_epochs, callbacks=get_callbacks('best_model'))
best_model.save('final_model')

Epoch 1/4
368/416 [=========================>....] - ETA: 0s - loss: 162.4254 - mse: 160.5420
Epoch: 0, loss:160.9637,  mse:159.0853,  
416/416 [==============================] - 1s 987us/step - loss: 160.9637 - mse: 159.0853
Epoch 2/4
416/416 [==============================] - 0s 1ms/step - loss: 156.1774 - mse: 154.3605
Epoch 3/4
416/416 [==============================] - 0s 1ms/step - loss: 154.1140 - mse: 152.3093
Epoch 4/4
416/416 [==============================] - 0s 1ms/step - loss: 152.7461 - mse: 150.9334
INFO:tensorflow:Assets written to: final_model\assets


In [16]:
test_results = best_model.evaluate(X_test, y_test)
dict(zip(best_model.metrics_names, test_results))

104/104 [==============================] - 0s 727us/step - loss: 155.3103 - mse: 153.4928


{'loss': 155.3102569580078, 'mse': 153.49281311035156}

In [19]:
pred = best_model.predict(X_test)
pred - np.array(y_test)

array([[ -1.585, -10.585,  19.415, ...,  13.415,   4.415, -14.585],
       [ -4.278, -13.278,  16.722, ...,  10.722,   1.722, -17.278],
       [-11.098, -20.098,   9.902, ...,   3.902,  -5.098, -24.098],
       ...,
       [-17.083, -26.083,   3.917, ...,  -2.083, -11.083, -30.083],
       [-14.909, -23.909,   6.091, ...,   0.091,  -8.909, -27.909],
       [-16.07 , -25.07 ,   4.93 , ...,  -1.07 , -10.07 , -29.07 ]])

In [28]:
def model_evaluation(y_pred, y_actual):
    """
    returns evaluation metrics for the model:
    Accuracy, F1 Score, R Squre, RMSe
    """
    acc = 0
    f1score = 0
    rmse = 0
    r_sqaure = 0
    
    if len(y_pred) != len(y_actual):
        print('predicted and actual length not equal')
        
    else:
        len_y = len(y_pred)
        
        y_pred_bool =  y_pred >= 0
        y_actual_bool =  y_actual >= 0
        f1score = f1_score(y_actual_bool, y_pred_bool, average='binary')
        acc = accuracy_score(y_actual_bool, y_pred_bool)
        
        rmse = np.sqrt(mse(y_actual, y_pred))
        r_sqaure = r2_score(y_actual, y_pred)
        
    df_evaluation = pd.DataFrame({'Accuracy': pd.Series(acc),
                                 'F1 Score': pd.Series(f1score),
                                 'R Square': pd.Series(r_sqaure),
                                 'RMSE': pd.Series(rmse)})
    return(df_evaluation)

In [29]:
model_evaluation(pred, np.array(y_test))

,Accuracy,F1 Score,R Square,RMSE
0,0.629396,0.703106,0.109798,12.389223
